In [1]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import sys   
from selenium.webdriver import Keys, ActionChains
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options as ChromeOptions
import time
import os    
import pandas as pd
import datetime as dt
import calendar as cl
import sys
sys.path.insert(1,'../../src')
import utils
from browse_twitter import get_tweets
# config= pd.read_csv(f'../../bucket/config/{utils.setConfig.select_data_folder()}',index_col=0)

In [ ]:
exact_search_term='iphone 15'
date = '9/15/2023'
months=['Jan ','Feb ','Mar ','Apr ','May ','Jun ','Jul ','Aug ','Sep ','Oct ','Nov ','Dec ']
config= pd.read_csv('../../bucket/config/iPhone15Config.csv',index_col=0)
phone_model,date=exact_search_term,gt.analyze_date_string(date)
day_after = date+dt.timedelta(days=7)
year,month,day=date.year,date.month,date.day
day_after_year,day_after_month,day_after_day=day_after.year,day_after.month,day_after.day
scan_summary = pd.read_csv(config.loc['scan_summary'][0])
all_tweets_repo = pd.read_csv(config.loc['tweets_repository'][0])
today = dt.datetime.now().strftime('%Y %m %d | %H:%M')
repo=[]
time_start=time.time()

In [ ]:
# To input your user credentials so you can use it in code

# os.environ['email']=
# os.environ['username']=
# os.environ['password']=
# os.environ['unexpected_twitter_login_code']=

In [2]:
'''
opens a chrome instance. Note that local chrome version and the chrome driver
in the working directory should match. Also note that selenium webscraping 
is finicky and needs continuous updating due to code changes or front end changes 
on the website being scraped. 
'''
chrome_options = ChromeOptions()
# chrome_options.add_experimental_option("detach", True) # to sometimes help with chrome tab stay open 
# chrome_options.add_argument("--headless=new") # so chrome doesn't open every single time
# chrome_options.add_experimental_option("debuggerAddress","localhost:9014") # alternative way to decalre port for created profile
# chrome_options.add_argument("--remote-debugging-port=9014")#opens created profile not to login again
cwd=os.getcwd()
chrome_options.add_argument(f'--user-data-dir={cwd}/tw_profile')#better change to absolute path if encountered errors # this line opens your profile which is already logged into twitter
service = Service(executable_path="../../chromedriver") # this is based on how this github repo is structured (i.e. chromedirver is in main directory alongside README.md)
driver = webdriver.Chrome(service=service,options=chrome_options)
driver.get('https://twitter.com')

In [ ]:
eml = os.environ.get('email')
usr=os.environ.get('username')
psw=os.environ.get('password')
unq=os.environ.get('unexpected_twitter_login_code')

In [ ]:
# driver = get_tweets.initiate_chrome()

In [ ]:
# get_tweets.login_to_twitter(driver,eml,usr,psw,unq)

In [4]:
get_tweets.find_tweets(driver,'9/15/2023',exact_search_term='iPhone 15',threshold=200,num_days=7,minRetweets=5)

1.0% has been done
1.0% has been done
4.0% has been done
7.000000000000001% has been done
12.0% has been done
17.0% has been done
21.0% has been done
25.0% has been done
28.999999999999996% has been done
33.0% has been done
37.0% has been done
41.0% has been done
45.0% has been done
49.0% has been done
53.0% has been done
56.99999999999999% has been done
62.0% has been done
67.0% has been done
70.0% has been done
74.0% has been done
79.0% has been done
81.0% has been done
84.0% has been done
89.0% has been done
93.0% has been done
97.0% has been done


In [ ]:
'''
loads in user credentials and logs into twitter
'''

eml = os.environ.get('email')
usr=os.environ.get('username')
psw=os.environ.get('password')
unq=os.environ.get('unexpected_twitter_login_code')
driver.implicitly_wait(15)
# login = driver.find_element(By.XPATH, '//span[contains(text(),"Log in")]')
# login.click()
time.sleep(2) # for popup to load cause it can be unreliable
popup=driver.find_element(By.XPATH,"//div[@aria-labelledby='modal-header' and @role='dialog' and contains(.,'Sign in')]")
username = driver.find_element(By.CSS_SELECTOR, "input[autocomplete='username']")
username.clear()
username.send_keys(eml)
username.send_keys(Keys.ENTER) #change this selecting next button #betterENG
time.sleep(2) #for selenium to update what popup is
if 'There was unusual' in popup.text:
    input=driver.find_element(By.CSS_SELECTOR,'input[name="text"]')#betterENG
    input.send_keys(usr)
    input.send_keys(Keys.ENTER)
password=driver.find_element(By.CSS_SELECTOR,'input[name="password"]')
password.send_keys(psw)
driver.find_element(By.CSS_SELECTOR,'div[role="button"][data-testid="LoginForm_Login_Button"]').click()
time.sleep(2)
print('Logged into twitter')

In [ ]:
#scrapes twitter
time_start=time.time()    
driver.get('https://twitter.com')
time.sleep(2)
explore=driver.find_element(By.CSS_SELECTOR,'a[href="/explore"][aria-label="Search and explore"]')
#width of the ec2 instance is 700px while height is 400px
if explore.size!=0:
    # print(explore.size,explore.is_displayed())
    explore.click()
    input = driver.find_element(By.CSS_SELECTOR,'input[aria-label="Search query"]')
else:
    input = driver.find_element(By.CSS_SELECTOR,'input[aria-label="Search query"]')
advanced_search=f'"{phone_model}" min_retweets:5 (lang:en) until:{day_after_year}-{day_after_month}-{day_after_day} since:{year}-{month}-{day}'
input.send_keys(advanced_search)
input.send_keys(Keys.ENTER)
latest = driver.find_element(By.CSS_SELECTOR,'a[href^="/search?q"][role="tab"][href$="f=live"]')
latest.click()
y_0=driver.execute_script("return document.body.scrollHeight")
count=0
count2=1
threshold=500# this would be a function input
if_no_articles=bool(len(driver.find_elements(By.XPATH,f'//div[contains(.,"No results for") and contains(.,"Try searching for something else")]')))
if if_no_articles==False:
    while count2<=threshold:
        print('{}% has been done'.format(round(count2/threshold,2)*100))
        tweet_elements=driver.find_elements(By.TAG_NAME,'article')
        for i in range(len(tweet_elements)):
            if tweet_elements[i].text=='This Tweet is unavailable.':
                continue
            text=''
            elements=tweet_elements[i].text.split('\n')
            if '·' not in elements[:4]:
                continue
            else:
                elements.remove('·')
                for k in range(1,len(elements)):
                        if elements[k][:4] in months:
                            date_index= k
                            break
                for j in range(date_index+1,len(elements)):
                    text+=elements[j]    
                row = elements[:date_index+1]
                row[date_index]=tweet_elements[i].find_element(By.TAG_NAME,'time').get_attribute('datetime')[:10]
                tw_date= dt.datetime.strptime(row[date_index],'%Y-%m-%d').strftime("%b %d, %Y")
                if len(row)==2:
                    row.insert(0,'No Name')    
                row.append(text)
                if row not in repo:
                    repo.append(row)
                    count2+=1
            if count2==threshold:
                break
        count+=1
        driver.execute_script("window.scrollTo(0, {});".format(count*y_0))
        time.sleep(3)
        if_pls_retry=bool(len(driver.find_elements(By.XPATH,f'//div[contains(.,"Something went wrong. Try reloading") and contains(.,"Retry")]')))
        if if_pls_retry:
            time.sleep(1800)
            driver.find_element(By.XPATH,f'//div[@role="button" and contains(.,"Retry")]').click()
        y_1= driver.execute_script("return document.body.scrollHeight")
        if y_1==y_0:
            time_of_break = time.time()
            break
        y_0=y_1       
else:
    scan_summary.loc[scan_summary.shape[0]]=['success/no results',today,time.time()-time_start,'None']
    scan_summary.head()
    scan_summary.to_csv(config.loc['iphone6_scan_summary'][0])            
    print('break')
    

**Note**
* in 3 hours about 5000 english tweets were captured globably that specifically mentioned iphone6. This number is really large
* Every 800 tweets or so the scrolling feature in twitter stops working for 5-10 mins. This slows down the scan
* Due to latest tab being choronologically ordered, and the large number of tweets, only tweets during night time could be captured after an hour of scraping (9pm to 12pm)

*As a result, only 500 tweets for each day is going to be grabbed for better scrape speed*

In [ ]:
for item in config.index.tolist():
    df=pd.read_csv(config.loc[item][0])
    df.head(0).to_csv(config.loc[item][0],index=False)

In [ ]:
repo_df=pd.DataFrame(repo,columns=['name','id','date','text'])
all_tweets_repo = pd.concat([all_tweets_repo,repo_df],ignore_index=True)
# all_tweets_repo.to_csv(config.loc['iphone6_tweets_repository'][0],index=False)
all_tweets_repo

In [ ]:
time_end=time.time()
process_runtime=time_end-time_start
scan_summary.loc[scan_summary.shape[0]]= ['success',today,process_runtime,'None']
scan_summary.to_csv(config.loc['iphone6_scan_summary'][0],index=False)